In [1]:
import qiskit
import cirq
import braket
import numpy as np
from qbraid.transpiler2.transpiler import qbraid_wrapper

In [2]:
# construct a regular qiskit circuit
qiskit_circ = qiskit.QuantumCircuit(4)

qiskit_circ.h([0, 1, 2, 3])
qiskit_circ.x([0, 1])
qiskit_circ.y(2)
qiskit_circ.z(3)
qiskit_circ.s(0)
qiskit_circ.sdg(1)
qiskit_circ.t(2)
qiskit_circ.tdg(3)
qiskit_circ.rx(np.pi/4, 0)
qiskit_circ.ry(np.pi/2, 1)
qiskit_circ.rz(3*np.pi/4, 2)
qiskit_circ.p(np.pi/8, 3)
qiskit_circ.sx(0)
qiskit_circ.sxdg(1)
qiskit_circ.iswap(2, 3)
qiskit_circ.swap([0, 1], [2, 3])
qiskit_circ.cx(0, 1)
qiskit_circ.cp(np.pi/4, 2, 3)

qiskit_circ.draw()

┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  ISWAP │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ TDG ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘

In [3]:
# Now let's make the same cirq circuit
cirq_circuit = cirq.Circuit()

# here the qubit ordering is normal, from 0 up to 3
q0 = cirq.LineQubit(0)
q1 = cirq.LineQubit(1)
q2 = cirq.LineQubit(2)
q3 = cirq.LineQubit(3)

cirq_gates = [
    cirq.H(q0),
    cirq.H(q1),
    cirq.H(q2),
    cirq.H(q3),
    cirq.X(q0),
    cirq.X(q1),
    cirq.Y(q2),
    cirq.Z(q3),
    cirq.S(q0),
    cirq.ZPowGate(exponent=-0.5)(q1),
    cirq.T(q2),
    cirq.ZPowGate(exponent=-0.25)(q3),
    cirq.rx(rads=np.pi / 4)(q0),
    cirq.ry(rads=np.pi / 2)(q1),
    cirq.rz(rads=3 * np.pi / 4)(q2),
    cirq.ZPowGate(exponent=1 / 8)(q3),
    cirq.XPowGate(exponent=0.5)(q0),
    cirq.XPowGate(exponent=-0.5)(q1),
    cirq.ISWAP(q2, q3),
    cirq.SWAP(q0, q2),
    cirq.SWAP(q1, q3),
    cirq.CNOT(q0, q1),
    cirq.CZPowGate(exponent=0.25)(q2, q3),
]

for gate in cirq_gates:
    cirq_circuit.append(gate)

# visually, this circuit appears the same as the original qiskit circuit
print(cirq_circuit)

                                          ┌──┐
0: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                           │     │
1: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           ││
2: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@────────
                                 │          │    │
3: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@^0.25───
                                          └──┘


In [4]:
# Now let's try transpiling a cirq circuit to qiskit. Transpiling the non-flipped cirq circuit to qiskit 
# should result in the same qiskit circuit and matrix representation as the original. 
qbraid_circuit_1 = qbraid_wrapper(cirq_circuit)
qiskit_circuit_transpiled = qbraid_circuit_1.transpile("qiskit")
qiskit_unitary_transpiled = qiskit.quantum_info.Operator(qiskit_circuit_transpiled).data

qiskit_circuit_transpiled.draw()

┌───┐┌───┐   ┌───┐   ┌─────────┐┌─────────┐                  
q0_0: ┤ H ├┤ X ├───┤ S ├───┤ RX(π/4) ├┤ Unitary ├─X─────────■──────
      ├───┤├───┤┌──┴───┴──┐├─────────┤├─────────┤ │       ┌─┴─┐    
q0_1: ┤ H ├┤ X ├┤ P(-π/2) ├┤ Unitary ├┤ Unitary ├─┼──X────┤ X ├────
      ├───┤├───┤└──┬───┬──┘├─────────┤└┬────────┤ │  │ ┌──┴───┴───┐
q0_2: ┤ H ├┤ Y ├───┤ T ├───┤ P(3π/4) ├─┤0       ├─X──┼─┤0         ├
      ├───┤├───┤┌──┴───┴──┐└┬────────┤ │  ISWAP │    │ │  Unitary │
q0_3: ┤ H ├┤ Z ├┤ P(-π/4) ├─┤ P(π/8) ├─┤1       ├────X─┤1         ├
      └───┘└───┘└─────────┘ └────────┘ └────────┘      └──────────┘